<a href="https://colab.research.google.com/github/irrundle3/comp646-plagiarism-checker/blob/main/integrity_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import torch

tokenizer = AutoTokenizer.from_pretrained('roberta-large')
model = AutoModel.from_pretrained('AIDA-UPM/star')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [3]:

drive.mount('/content/drive')

# Access CSV file
file_path = '/content/drive/My Drive/blogtext.csv'




Mounted at /content/drive


In [4]:
blog_data = pd.read_csv(file_path)

n_values = len(blog_data.id.unique())
author_mapping = {k: v for k, v in zip(blog_data.id.unique(), range(n_values))}

blog_data['id'] = blog_data['id'].apply(lambda x: 'blog_' + str(author_mapping[x]))

blog_data



,id,gender,age,topic,sign,date,text
0,blog_0,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,blog_0,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,blog_0,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,blog_0,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,blog_1,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
681279,blog_19319,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,blog_19319,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,blog_19319,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,blog_19319,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


In [5]:
filtered_df = blog_data[blog_data['id'].str[5:].astype(int) <= 50]
# filtered_df.iloc[76]
filtered_df

,id,gender,age,topic,sign,date,text
0,blog_0,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,blog_0,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,blog_0,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,blog_0,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,blog_1,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
1384,blog_49,male,23,Student,Sagittarius,"25,July,2004",Let's see if this is any good... ...
1385,blog_49,male,23,Student,Sagittarius,"16,August,2004",urlLink Evolutionism Propaganda Y...
1386,blog_50,female,24,Banking,Libra,"16,August,2004","Hello, I’m back again………. Friday ..."
1387,blog_50,female,24,Banking,Libra,"03,August,2004",12th August After a long thought...


In [6]:
result_df = filtered_df
model.to('cuda')

def find_similarity(texts):
    encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    tensor_input_ids = torch.tensor(encoded_texts['input_ids']).clone().detach().to('cuda')  # Move tensor to GPU
    tensor_attention_mask = torch.tensor(encoded_texts['attention_mask']).clone().detach().to('cuda')  # Move tensor to GPU

    with torch.no_grad():
        style_embeddings = model(input_ids=tensor_input_ids, attention_mask=tensor_attention_mask).pooler_output

    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-8)
    return cos(style_embeddings[0].cpu(), style_embeddings[1].cpu())  # Move tensors back to CPU before using them with NumPy

def apply_function(row, result_df):
    result_df = result_df[result_df['id'] != row['id']]
    sampled_rows = result_df.sample(n=5, replace=True)
    result = sampled_rows.apply(lambda sampled_row: find_similarity([row['text'],sampled_row['text']]), axis=1).mean()
    return result

result_df['Result'] = result_df.apply(lambda row: apply_function(row, result_df), axis=1)
result_df


<ipython-input-6-56f6627a5adf>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_input_ids = torch.tensor(encoded_texts['input_ids']).clone().detach().to('cuda')  # Move tensor to GPU
<ipython-input-6-56f6627a5adf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_attention_mask = torch.tensor(encoded_texts['attention_mask']).clone().detach().to('cuda')  # Move tensor to GPU
<ipython-input-6-56f6627a5adf>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_input_ids = torch.tensor(encoded_texts['input_ids']).clone().detach

,id,gender,age,topic,sign,date,text,Result
0,blog_0,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",0.523064
1,blog_0,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,0.509839
2,blog_0,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,0.547655
3,blog_0,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,0.486411
4,blog_1,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,0.586336
...,...,...,...,...,...,...,...,...
1384,blog_49,male,23,Student,Sagittarius,"25,July,2004",Let's see if this is any good... ...,0.566146
1385,blog_49,male,23,Student,Sagittarius,"16,August,2004",urlLink Evolutionism Propaganda Y...,0.596522
1386,blog_50,female,24,Banking,Libra,"16,August,2004","Hello, I’m back again………. Friday ...",0.533942
1387,blog_50,female,24,Banking,Libra,"03,August,2004",12th August After a long thought...,0.498906


In [7]:


def apply_function_within(row,result_df):
    result_df = result_df[result_df['id'] == row['id']]
    result = result_df.apply(lambda sampled_row: find_similarity([row['text'],result_df['text']]), axis=1).mean()
    return result


result_df['within_result'] = result_df.apply(lambda row: apply_function(row, result_df), axis=1)
result_df

<ipython-input-6-56f6627a5adf>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_input_ids = torch.tensor(encoded_texts['input_ids']).clone().detach().to('cuda')  # Move tensor to GPU
<ipython-input-6-56f6627a5adf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_attention_mask = torch.tensor(encoded_texts['attention_mask']).clone().detach().to('cuda')  # Move tensor to GPU
<ipython-input-7-dc66cab26ef8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

,id,gender,age,topic,sign,date,text,Result,within_result
0,blog_0,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",0.523064,0.502961
1,blog_0,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,0.509839,0.538905
2,blog_0,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,0.547655,0.585070
3,blog_0,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,0.486411,0.498971
4,blog_1,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,0.586336,0.553298
...,...,...,...,...,...,...,...,...,...
1384,blog_49,male,23,Student,Sagittarius,"25,July,2004",Let's see if this is any good... ...,0.566146,0.569230
1385,blog_49,male,23,Student,Sagittarius,"16,August,2004",urlLink Evolutionism Propaganda Y...,0.596522,0.545047
1386,blog_50,female,24,Banking,Libra,"16,August,2004","Hello, I’m back again………. Friday ...",0.533942,0.428950
1387,blog_50,female,24,Banking,Libra,"03,August,2004",12th August After a long thought...,0.498906,0.486471


In [8]:
# for id_value, group_data in filtered_df:
#     # Extract the blog inputs for this group
#     blog_inputs = group_data.index.tolist()

#     # # Generate pairs of inputs for the function
#     # for i, input1 in enumerate(blog_inputs):
#     #     for input2 in blog_inputs[i+1:]:
#     #         your_function(input1, input2)
mean_A = result_df['within_result'].mean()
mean_B = result_df['Result'].mean()

print(mean_A)
print(mean_B)


0.5232043188196708
0.5245259299219899
